In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import sqlite3

conn = sqlite3.connect("database_raw.db")
cursor = conn.cursor()

folder_path ="./Dataset"

# Lấy danh sách tất cả các file CSV trong folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Đọc tất cả các file CSV vào danh sách DataFrame
# dfs = [pd.read_csv(file) for file in csv_files]

for file in csv_files:
    print(f"Xử lý {file}")
    
    chunk_size = 1000000
    for chunk in pd.read_csv(file, chunksize=chunk_size):
        chunk.to_sql("raw_table", conn, if_exists = "append", index=False)
    
# # Nếu muốn gộp tất cả các file CSV thành một DataFrame duy nhất
# combined_df = pd.concat(dfs, ignore_index=True)

Xử lý ./Dataset\part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00012-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Xử lý ./Dataset\part-00013-363d1ba3-8ab5-4f96-bc25-

In [3]:
query = "SELECT * FROM raw_table LIMIT 10;"
df1= pd.read_sql_query(query, conn)
print (df1)
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import sqlite3
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

def process_data(combined_df):
    # y = np.array(yi).reshape(-1, 1)
    y_raw = combined_df.iloc[:, -1].values
    y = np.array(y_raw).reshape(-1, 1)

    attack_types =["Benign", "Backdoor", "Hijacking", "Injection", "DDoS",  "DoS", 
                            "Mirai", "Spoofing", "Recon", "XSS"]

    label_mapping = {}

    # Init
    encode = OneHotEncoder(sparse_output=False)

    # Mã hóa dữ liệu
    y_encoded = encode.fit_transform(y)

    # lấy list label
    labels = encode.categories_[0]

    ## mapping
    for label in labels:
        label_mapping[label]= -1
        for inx, type in enumerate(attack_types):
            if label.find(type)!=-1:
                label_mapping[label] = inx
                break

    # print(label_mapping)

    for i, value in enumerate(combined_df['label']):
        combined_df.at[i, 'label'] = label_mapping.get(value, -1)
        
    # loại bỏ những hàng mà cột label chứa giá trị -1
    combined_df = combined_df[combined_df['label'] != -1]

    # kiểm tra có còn hàng nào chứa -1 k
    # print(combined_df[combined_df['label'] == -1].index)

    # print(combined_df.head())
    # print("------------------------\n")
    
    # sau khi onehotencoder
    # y_final_encoded = encode.fit_transform(combined_df[['label']])

    # # Thêm vào DataFrame
    # encoded_labels = [f'label_{int(i)}' for i in encode.categories_[0]]
    # combined_df_encoded = pd.DataFrame(y_final_encoded, columns=encoded_labels)

    # # Nối với DataFrame gốc (bỏ cột `label` cũ)
    # combined_df = pd.concat([combined_df.reset_index(drop=True), combined_df_encoded], axis=1).drop(columns=['label'])
    
    # print(combined_df['Radius'].head())
    
    return combined_df


In [2]:
conn = sqlite3.connect("database_raw.db")
new_conn = sqlite3.connect("C:/Users/hoang/Data_Train_2025/database_labeled_all.db")
cursor = conn.cursor()

batch_size = 100000
cursor.execute("SELECT * FROM raw_table;")


while True:
    rows = cursor.fetchmany(batch_size)
    if not rows:
        break
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    
    df = process_data(df)
    
    df.to_sql("one_hot_encoded_data", new_conn, if_exists="append", index=False)

conn.close()
new_conn.close()